# **Frames**

This notebook demonstrates how to use **SigmaEpsilon** to calculate 3d frames, and provides with some comparison to other parametric modelling frameworks.

## Input Data

In [1]:
Lx, Ly, Lz = 40., 12., 8.
nx, ny, nz = 8, 4, 4

D = 0.1  # outer diameter of the tube
t = 0.05  # thickness of the tube
F = 100.0

nCase = 15


In [2]:
from polymesh.space import StandardFrame, PointCloud
from polymesh.grid import gridH8 as grid
from polymesh.utils.topology.tr import H8_to_L2
import numpy as np

# mesh
gridparams = {
    'size': (Lx, Ly, Lz),
    'shape': (8, 4, 4),
    'origo': (0, 0, 0),
    'start': 0
}

coords, topo = grid(**gridparams)
coords, topo = H8_to_L2(coords, topo)

GlobalFrame = StandardFrame(dim=3)

points = PointCloud(coords, frame=GlobalFrame).centralize()
dx = - np.array([points[:, 0].min(), 0., 0.])
points.move(dx)
coords = points.show()


In [3]:
from linkeddeepdict import LinkedDeepDict
from time import time

summary = LinkedDeepDict()


## AxisVM

In [4]:
from axisvm.com.client import start_AxisVM
axvm = start_AxisVM(visible=True, daemon=True)
axvm


ModuleNotFoundError: No module named 'sigmaepsilon.core'

In [ ]:
import axisvm.com.tlb as axtlb

# create new model
modelId = axvm.Models.New()
axm = axvm.Models[modelId]

# material
ndc, material = axtlb.ndcEuroCode, "S 235"
axm.Settings.NationalDesignCode = ndc

# cross section
MaterialIndex = axm.Materials.AddFromCatalog(ndc, material)
CrossSectionIndex = axm.CrossSections.AddCircleHollow('S1', D, t)

# crate nodes
fnc = axm.Nodes.Add
list(map(lambda c: fnc(*c), coords))

# create lines
fnc = axm.Lines.Add
GeomType = axtlb.lgtStraightLine
list(map(lambda x: fnc(x[0], x[1], GeomType), topo + 1))

# set material and cross section
LineAttr = axtlb.RLineAttr(
    LineType=axtlb.ltBeam,
    MaterialIndex=MaterialIndex,
    StartCrossSectionIndex=CrossSectionIndex,
    EndCrossSectionIndex=CrossSectionIndex
)
lineIDs = [i+1 for i in range(axm.Lines.Count)]
attributes = [LineAttr for _ in range(axm.Lines.Count)]
axm.Lines.BulkSetAttr(lineIDs, attributes)

# essential boundary conditions
spring = axtlb.RStiffnesses(x=1e12, y=1e12, z=1e12, xx=1e12, yy=1e12, zz=1e12)
RNonLinearity = axtlb.RNonLinearity(
    x=axtlb.lnlTensionAndCompression,
    y=axtlb.lnlTensionAndCompression,
    z=axtlb.lnlTensionAndCompression,
    xx=axtlb.lnlTensionAndCompression,
    yy=axtlb.lnlTensionAndCompression,
    zz=axtlb.lnlTensionAndCompression
)
RResistances = axtlb.RResistances(x=0, y=0, z=0, xx=0, yy=0, zz=0)
ebcinds = np.where(coords[:, 0] < 1e-12)[0]
for i in ebcinds:
    axm.NodalSupports.AddNodalGlobal(spring, RNonLinearity, RResistances, i+1)

# natural boundary conditions
load_cases = {}
LoadCaseType = axtlb.lctStandard
inds = np.where(coords[:, 0] > Lx - 1e-12)[0] + 1
axm.BeginUpdate()
for case in range(nCase):
    name = 'LC{}'.format(case+1)
    lcid = axm.LoadCases.Add(name, LoadCaseType)
    pid = np.random.choice(inds)
    Fx, Fy, Fz = 0, 0, -F
    force = axtlb.RLoadNodalForce(
        LoadCaseId=lcid,
        NodeId=pid,
        Fx=Fx, Fy=Fy, Fz=Fz,
        Mx=0., My=0., Mz=0.,
        ReferenceId=0
    )
    axm.Loads.AddNodalForce(force)
    load_cases[lcid] = dict(name=name, id=case)
    summary['AxisVM', name] = LinkedDeepDict(lcid=lcid, fid=pid, f=[Fx, Fy, Fz])
axm.EndUpdate()

fpath = 'console_H8_L2.axs'
axm.SaveToFile(fpath, False)
summary['AxisVM', 'path'] = fpath
axm


In [ ]:
t0 = time()
axm.Calculation.LinearAnalysis(axtlb.cuiNoUserInteractionWithAutoCorrectNoShow)
dt = time() - t0
summary['AxisVM', 'proc', 'time'] = dt


In [ ]:
nIDs = [i+1 for i in range(axm.Nodes.Count)]
disps = axm.Results.Displacements
disps.DisplacementSystem = axtlb.dsGlobal
for lcid, lcdata in load_cases.items():
    lcname = lcdata['name']
    dofsol_case = np.zeros((axm.Nodes.Count, 3))
    disps.LoadCaseId = lcid
    disps.LoadLevelOrModeShapeOrTimeStep = 1
    def fnc(i): return disps.NodalDisplacementByLoadCaseId(i)[0].ex
    dofsol_case[:, 0] = np.array(list(map(fnc, nIDs)))
    def fnc(i): return disps.NodalDisplacementByLoadCaseId(i)[0].ey
    dofsol_case[:, 1] = np.array(list(map(fnc, nIDs)))
    def fnc(i): return disps.NodalDisplacementByLoadCaseId(i)[0].ez
    dofsol_case[:, 2] = np.array(list(map(fnc, nIDs)))
    summary['AxisVM', lcname, 'dofsol'] = np.copy(dofsol_case)


In [ ]:
axm.View = axtlb.vFront
axvm.MainFormTab = axtlb.mftGeometry
axm.FitInView()
axm.Windows[1].screenshot()


In [ ]:
# we want the domain to fill up the screen entirely
axm.View = axtlb.vFront
axvm.MainFormTab = axtlb.mftStatic
axm.FitInView()

# turn off the grid
GridOptions = axtlb.RGridOptions(DisplayGrid=False)
axm.Settings.SetGridOptions(GridOptions)

WriteValuesTo = axtlb.RWriteValuesTo(
    Nodes=True,
    Lines=False,
    Surfaces=False,
    MinMaxOnly=True
)

BasicDispParams = axtlb.RBasicDisplayParameters_V153(
    ResultComponent=axtlb.rc_d_eZ,
    Scale=1.0,
    DisplayMode=axtlb.dmDiagram,
    DisplayShape=axtlb.dsDeformed,
    WriteValuesTo=WriteValuesTo
)

ExtDispParams = axtlb.RExtendedDisplayParameters_V153(
    BasicDispParams=BasicDispParams,
    DisplayAnalysisType=axtlb.datLinear,
    ResultType=axtlb.rtLoadCombination
)

axm.Windows.SetStaticDisplayParameters_V153(1, ExtDispParams, 2, [])

axm.Windows.ReDraw()
axvm.BringToFront()


In [ ]:
axm.Windows[1].screenshot()


## SigmaEpsilon

In [ ]:
from sigmaepsilon import Structure, LineMesh, PointData
from polymesh.space import StandardFrame
from polymesh.utils.space import frames_of_lines
from sigmaepsilon.fem.cells import B2 as Beam
import numpy as np

# model stiffness matrix
material = axm.Materials.Item[MaterialIndex]
E, nu = material.Ex, material.Nux
section = axm.CrossSections.Item[CrossSectionIndex]
A, Iy, Iz, Ix = section.Ax, section.Iy, section.Iz, section.Iyz
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

# essential boundary conditions
ebcinds = np.where(coords[:, 0] < 1e-12)[0]
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
fixity[ebcinds, :] = True

# natural boundary conditions
loads = np.zeros((coords.shape[0], 6, nCase))
for lcdata in load_cases.values():
    lcname = lcdata['name']
    id = lcdata['id']
    fid = summary['AxisVM', lcname, 'fid'] - 1
    f = summary['AxisVM', lcname, 'f']
    loads[fid, :len(f), id] = f
loads = loads.reshape(coords.shape[0] * 6, nCase)

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, material=Hooke, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, frame=GlobalFrame)
structure = Structure(mesh=mesh)


In [ ]:
t0 = time()
structure.linsolve(summary=True)
dt = time() - t0
summary['SigEps', 'proc', 'time'] = dt


In [ ]:
dofsol = structure.nodal_dof_solution()[:, :3, :]

for lcdata in load_cases.values():
    lcname = lcdata['name']
    id = lcdata['id']
    summary['SigEps', lcname, 'dofsol'] = dofsol[:, :, id]


In [ ]:
from dewloosh.core.tools import float_to_str_sig
from neumann import minmax
def f4(v): return float_to_str_sig(v, sig=4)


load_case_names = list(load_cases.values())

diffs = []
for lcdata in load_cases.values():
    lcname = lcdata['name']
    id = lcdata['id']
    r_axvm = summary['AxisVM', lcname, 'dofsol'].min()
    r_sigeps = summary['SigEps', lcname, 'dofsol'].min()
    diffp = 100 * np.abs(r_axvm - r_sigeps) / \
        np.min(np.abs([r_axvm, r_sigeps]))
    diffs.append(diffp)
print('min : {} % \nmax : {} %'.format(
    *list(map(f4, minmax(np.array(diffs))))))


In [ ]:
t_sigeps = summary['SigEps', 'proc', 'time']
t_axvm = summary['AxisVM', 'proc', 'time']
t_axvm / t_sigeps
